# Psi4 Notebook for educational purposes


This notebook is used to compare the results of Psi4 with the results with my own implementation of the Hartree-Fock method. It helps me to check if my implementation is correct. 

Connect to pc with `Psi4` installation and host jupyter server.

`ssh -L 8080:localhost:8080 username@pcname`

`jupyter notebook --no-browser --port=8080`

In [23]:
ls

eri_Psi4_test.dat  h2o_test.py*    h2o_testV2.py*        psi4_setup_h2.py
h2o_test.log       h2o_testV2.log  ipython_h2o_setup.py  timer.dat
h2o_test.out       h2o_testV2.out  output.dat


My setup, that is run:

In [24]:
cat psi4_setup_h2.py

import psi4
import numpy as np 

np.set_printoptions(precision=5, linewidth=100, suppress=True)
psi4.set_memory("4 GB")
psi4.core.set_output_file('output.dat', False)
numpy_memory = 4

mol = psi4.geometry("""
0 1
H 0.0 0.0 0.0
H 0.0 0.0 1.4
units bohr
""")

psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "sto-3g", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8})

wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())

S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())

no_basis_funcs: int = S.shape[0]



In [25]:
%run psi4_setup_h2.py

### This is with STO-3G basis set 

In [26]:
ERI

array([[[[0.77461, 0.44411],
         [0.44411, 0.56968]],

        [[0.44411, 0.29703],
         [0.29703, 0.44411]]],


       [[[0.44411, 0.29703],
         [0.29703, 0.44411]],

        [[0.56968, 0.44411],
         [0.44411, 0.77461]]]])

### This is with 6-311G basis set 

In [27]:
psi4.set_options({"save_jk": "true", "guess": "CORE", "basis": "6-311G", "DIIS": "false", "SCF_INITIAL_ACCELERATOR": "None", "CFOUR_SCF_MAXCYC": "500", "e_convergence": 1e-8})

In [28]:
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option("BASIS"))
mints = psi4.core.MintsHelper(wfn.basisset())
S = np.asarray(mints.ao_overlap())
T = np.asarray(mints.ao_kinetic())
V = np.asarray(mints.ao_potential())
ERI = np.asarray(mints.ao_eri())